In [1]:
import pandas as pd

df = pd.read_csv('../data/2018-11-west-midlands-street.csv').dropna(subset=['Longitude', 'Latitude'])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25938 entries, 0 to 25937
Data columns (total 12 columns):
Crime ID                 22805 non-null object
Month                    25938 non-null object
Reported by              25938 non-null object
Falls within             25938 non-null object
Longitude                25938 non-null float64
Latitude                 25938 non-null float64
Location                 25938 non-null object
LSOA code                25936 non-null object
LSOA name                25936 non-null object
Crime type               25938 non-null object
Last outcome category    22805 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 2.6+ MB
None


In [2]:
df['Coordinate'] = df.apply(lambda x: (x['Longitude'], x['Latitude']), axis=1)

df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Coordinate
0,0f3bd1cfb66788cef7ef1cc3432245cf0152296db62a62...,2018-11,West Midlands Police,West Midlands Police,-1.850056,52.591108,On or near Walsall Road,E01009417,Birmingham 001A,Burglary,Investigation complete; no suspect identified,NaN,"(-1.8500560000000001, 52.591108)"
1,074eb80647c8f79f1fd4203c222d300cc493ef6e4d5881...,2018-11,West Midlands Police,West Midlands Police,-1.850519,52.589275,On or near Crown Lane,E01009417,Birmingham 001A,Vehicle crime,Investigation complete; no suspect identified,NaN,"(-1.8505189999999998, 52.589275)"
2,f72a479509efce158790f541ea8377123da3866691af0f...,2018-11,West Midlands Police,West Midlands Police,-1.849280,52.593346,On or near Russell Bank Road,E01009417,Birmingham 001A,Vehicle crime,Investigation complete; no suspect identified,NaN,"(-1.84928, 52.593346)"
3,NaN,2018-11,West Midlands Police,West Midlands Police,-1.845780,52.593827,On or near Hook Drive,E01009418,Birmingham 001B,Anti-social behaviour,NaN,NaN,"(-1.84578, 52.593827000000005)"
4,NaN,2018-11,West Midlands Police,West Midlands Police,-1.843524,52.597348,On or near Knightsbridge Close,E01009418,Birmingham 001B,Anti-social behaviour,NaN,NaN,"(-1.8435240000000002, 52.597348)"


In [3]:
crime_sr = df.groupby(['Coordinate', 'Crime type']).size()

In [4]:
crime_freq = df.groupby('Coordinate').size()

print(crime_freq.min())
print(crime_freq.max())

1
67


In [5]:
features = []

last_coord = None
data = None

for (coord, crime_type), crime_count in crime_sr.iteritems():
    
    # Create a new data if it's the new location
    if coord != last_coord:
        if data is not None:
            features.append(data)
                
        # Re-initialise the data
        data = {
            'type': 'Feature',
            'properties': {
                'totalCrimeCount': 0,
                'crimeCountPerType': {}
            },
            'geometry': {
                'type': 'Point',
                'coordinates': []
            }
        }
        
    data['properties']['totalCrimeCount'] += crime_count
    data['properties']['crimeCountPerType'][crime_type] = crime_count
    data['properties']['coordinates'] = f'{coord[1]}, {coord[0]}'
    data['geometry']['coordinates'] = [coord[0], coord[1]]
    
    last_coord = coord

In [6]:
features

[{'geometry': {'coordinates': [-7.5768059999999995, 54.546177],
   'type': 'Point'},
  'properties': {'coordinates': '54.546177, -7.5768059999999995',
   'crimeCountPerType': {'Criminal damage and arson': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-7.5709550000000005, 54.520635],
   'type': 'Point'},
  'properties': {'coordinates': '54.520635, -7.5709550000000005',
   'crimeCountPerType': {'Criminal damage and arson': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-5.097872, 50.026055], 'type': 'Point'},
  'properties': {'coordinates': '50.026055, -5.097872',
   'crimeCountPerType': {'Vehicle crime': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.718838, 50.206416], 'type': 'Point'},
  'properties': {'coordinates': '50.206416, -3.718838',
   'crimeCountPerType': {'Public order': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-3.683777999

In [7]:
import json

with open('./template.geojson', 'r') as template:
    template = json.load(template)
    template['features'] = features
    template['minCrimeCount'] = int(crime_freq.min())
    template['maxCrimeCount'] = int(crime_freq.max())

with open('2018-11-west-midlands-street.geojson', 'w') as output:
    json.dump(template, output)